# Running Faces pipeline

In [43]:
#!pip uninstall mlrun -y

#!pip install colorlog PyHive mlrun==0.5.1-rc1 kubernetes
!pip show mlrun

### create the mlrun project 

In [44]:
from os import path, getenv
from mlrun import new_project, mlconf

#project_name = '-'.join(filter(None, ['getting-started-iris', getenv('V3IO_USERNAME', None)]))
project_name = "faces"
project_path = path.abspath('./')
project = new_project(project_name, project_path)
project.save()
print(f'Project path: {project_path}\nProject name: {project_name}')

Project path: /User/mlrun/demos/faces/notebooks
Project name: faces


In [45]:
out = mlconf.artifact_path or path.abspath('./data')
# {{run.uid}} will be substituted with the run id, so output will be written to different directoried per run
artifact_path = path.join(out, '{{run.uid}}')
%env PYTHONPATH=./

env: PYTHONPATH=./


### set the project functions

In [46]:
from mlrun import mount_v3io
#project.set_function("functions/clean_parquez.py", 'clean', kind='job', image='aviaigz/parquez')
project.set_function("../functions/encode_images.py", 'encode_images', kind='job', image='ml-models/ml-models-gpu')
project.set_function("../functions/train.py", 'train', kind='job', image='ml-models/ml-models-gpu')
project.func('encode_images').apply(mount_v3io())
project.func('train').apply(mount_v3io())
project.func('encode_images').set_env('PYTHONPATH', project_path)
project.func('train').set_env('PYTHONPATH', project_path)
project.func('encode_images').spec.artifact_path = 'User/artifacts'
project.func('train').spec.artifact_path = 'User/artifacts'

<a id="gs-step-create-n-run-ml-pipeline"></a>
## Create and Run a Fully Automated ML Pipeline

You're now ready to create a full ML pipeline.
This is done by using [Kubeflow Pipelines](https://www.kubeflow.org/docs/pipelines/overview/pipelines-overview/), which is integrated into the Iguazio Data Science Platform.
Kubeflow Pipelines is an open-source framework for building and deploying portable, scalable machine-learning workflows based on Docker containers.
MLRun leverages this framework to take your existing code and deploy it as steps in the pipeline.

In [47]:
%%writefile {path.join(project_path, 'workflow.py')}

from kfp import dsl
from mlrun import mount_v3io
from os import path

funcs = {}
project_path = path.abspath('./')


# Configure function resources and local settings
def init_functions(functions: dict, project=None, secrets=None):
    project_path = path.abspath('./')
    for f in functions.values():
        f.apply(mount_v3io())
        f.set_env('PYTHONPATH', project_path)
        f.spec.artifact_path = 'User/artifacts'
        f.spec.service_account='mlrun-api'
        
        
# Create a Kubeflow Pipelines pipeline
@dsl.pipeline(
    name = "faces-pipeline",
    description = "faces demo pipeline"
)
def kfpipeline():

    # Ingest the data set
    validate = funcs['encode_images'].as_step(
        name="encode_images",
        #params=params,
#         inputs={'table': clean.outputs},
        outputs=['validate']
    )
    
    # Analyze the dataset
    schema = funcs['train'].as_step(
        name="train",
        #params = params,
        inputs={'table': validate.outputs},                       
        outputs=['schema']
    )

Overwriting /User/mlrun/demos/faces/notebooks/workflow.py


<a id="gs-register-workflow"></a>
#### Register the Workflow

Use the `set_workflow` MLRun project method to register your workflow with MLRun.
The following code sets the `name` parameter to the selected workflow name ("main") and the `code` parameter to the name of the workflow file that is found in your project directory (**workflow.py**).

In [48]:
# Register the workflow file as "main"
project.set_workflow('main', 'workflow.py')

In [49]:
project.save()

In [50]:
run_id = project.run(
    'main',
    arguments={}, 
    
    artifact_path=path.abspath(path.join('pipeline','{{workflow.uid}}'),
    
                              )
    ,dirty=True)

> 2020-10-11 10:09:39,147 [warning] warning!, server (0.5.1) and client (0.5.2) ver dont match
> 2020-10-11 10:09:39,261 [warning] warning!, server (0.5.1) and client (0.5.2) ver dont match


> 2020-10-11 10:09:39,627 [info] Pipeline run id=38416347-b241-487b-90fe-eaef826f3351, check UI or DB for progress
